In [7]:
import os
import glob
import tkinter as tk
from tkinter import filedialog
import struct
import pandas as pd
import re

def select_folder():
    """
    フォルダ選択ダイアログを表示し、選択されたフォルダのパスを返します。
    """
    root = tk.Tk()
    root.withdraw()  # GUIを隠す
    folder_path = filedialog.askdirectory()
    return folder_path

def find_d0_files(folder_path):
    """
    指定されたフォルダ内の全てのサブフォルダを再帰的に探索し、
    .d0または.D0を含む拡張子を持つファイルのパスをリストで返します。

    Args:
        folder_path (str): 検索対象のフォルダのパス
    """
    d0_files = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if re.search(r'\.[dD]\d+\.?\w*', file):  # .d0または.D0を含む拡張子を探す（.d00.tmpのような形式にも対応）
                d0_files.append(os.path.join(root, file))
    return d0_files

def load_sensor_data(file_path):
    """
    指定されたファイルパスからセンサデータを読み込み、
    各センサの値をリストとして返します。

    Args:
        file_path (str): センサデータファイルのパス
    """
    with open(file_path, 'rb') as file:
        data = file.read()

    sensor_data = [data[i:i+8*2] for i in range(408, len(data), 8*2)]
    sensor_values = [[] for _ in range(8)]
    for sensor in sensor_data:
        for i in range(8):
            sensor_value = unsigned2signed(int.from_bytes(sensor[i*2:i*2+2], 'little'))
            sensor_values[i].append(sensor_value)
    
    return sensor_values

def unsigned2signed(unsigned_value):
    """
    符号なし16ビット整数を符号付き16ビット整数に変換します。

    Args:
        unsigned_value (int): 符号なし16ビット整数
    """
    if unsigned_value >= 32768:
        return unsigned_value - 65536
    else:
        return unsigned_value

def write_to_excel(sensor_values, file_name, save_dir):
    """
    センサ値をExcelファイルに書き込みます。

    Args:
        sensor_values: センサ値のリスト
        file_name: 保存するファイル名
        save_dir: 保存するディレクトリ
    """
    excel_file_name = os.path.join(save_dir, f"{file_name}.xlsx")
    df = pd.DataFrame(sensor_values).T  # 転置して各センサーが列になるようにする
    df.columns = [f'Sensor_{i+1}' for i in range(8)]
    df.to_excel(excel_file_name, index=False)
    print(f"{excel_file_name} に書き出しました。")

def process_subfolders(base_folder, save_base_folder):
    """
    指定されたフォルダ内の全てのサブフォルダを処理します。

    Args:
        base_folder (str): 処理対象のフォルダのパス
        save_base_folder (str): Excelファイルを保存するフォルダのパス
    """
    subfolders = [f.path for f in os.scandir(base_folder) if f.is_dir()]
    for subfolder in subfolders:
        subfolder_name = os.path.basename(subfolder)
        save_dir = os.path.join(save_base_folder, subfolder_name)
        os.makedirs(save_dir, exist_ok=True)
        
        d0_files = find_d0_files(subfolder)
        if not d0_files:
            print(f"{subfolder} 内に拡張子に.d0を含むファイルが見つかりませんでした。")
            continue

        for file_path in d0_files:
            file_name = os.path.basename(file_path).split('.')[0]  # 拡張子を除いたファイル名
            sensor_values = load_sensor_data(file_path)
            write_to_excel(sensor_values, file_name, save_dir)

def main():
    base_folder = select_folder()
    if not base_folder:
        print("フォルダが選択されませんでした。")
        return

    save_base_folder = r"C:\Users\r-fujita\Desktop\ET\藤澤さんデータ\リスト3課\Data_Excel"  # 保存先フォルダ
    process_subfolders(base_folder, save_base_folder)

if __name__ == "__main__":
    main()

C:\Users\r-fujita\Desktop\ET\藤澤さんデータ\リスト3課\Data_Excel\NE-121A\001-002.xlsx に書き出しました。
C:\Users\r-fujita\Desktop\ET\藤澤さんデータ\リスト3課\Data_Excel\NE-121A\001-002.xlsx に書き出しました。
C:\Users\r-fujita\Desktop\ET\藤澤さんデータ\リスト3課\Data_Excel\NE-121A\002-001.xlsx に書き出しました。
C:\Users\r-fujita\Desktop\ET\藤澤さんデータ\リスト3課\Data_Excel\NE-121A\002-003.xlsx に書き出しました。
C:\Users\r-fujita\Desktop\ET\藤澤さんデータ\リスト3課\Data_Excel\NE-121A\002-006.xlsx に書き出しました。
C:\Users\r-fujita\Desktop\ET\藤澤さんデータ\リスト3課\Data_Excel\NE-121A\004-003.xlsx に書き出しました。
C:\Users\r-fujita\Desktop\ET\藤澤さんデータ\リスト3課\Data_Excel\NE-121A\004-008.xlsx に書き出しました。
C:\Users\r-fujita\Desktop\ET\藤澤さんデータ\リスト3課\Data_Excel\NE-121A\006-008.xlsx に書き出しました。
C:\Users\r-fujita\Desktop\ET\藤澤さんデータ\リスト3課\Data_Excel\NE-121A\007-002.xlsx に書き出しました。
C:\Users\r-fujita\Desktop\ET\藤澤さんデータ\リスト3課\Data_Excel\NE-121A\009-001.xlsx に書き出しました。
C:\Users\r-fujita\Desktop\ET\藤澤さんデータ\リスト3課\Data_Excel\NE-121A\009-004.xlsx に書き出しました。
C:\Users\r-fujita\Desktop\ET\藤澤さんデータ\リスト3課\Data_Excel\NE-121A\009